In [1]:
import pandas as pd
from tqdm.notebook import tqdm

# Подготовка статистики игроков


Таблица со всей статистикой игроков.

In [2]:
player_stats = pd.read_csv('/content/drive/MyDrive/xg_project/Players data 3.csv')
player_stats.replace(',','', regex=True, inplace=True) # В столбце с минутами тысячи отделены запятыми, удаляем их и кастим к флоату
player_stats['minutes'] = player_stats['minutes'].astype('float')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (195) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Функция возвращающая список таблиц с трансферами для каждого игрока


In [3]:
def get_seasons(player, table, max_seasons = 1, min_games_before = 5, min_games_after = 5):
  
  df = table[table['name'] == player].copy()
  df['year'] = df['season'].astype(str).str[:4]
  df['year'] = df['year'].astype('float64')
  is_keeper = (df.iloc[-1,:]['all_goals_against_gk'] == df.iloc[-1,:]['all_goals_against_gk'])
  if (is_keeper):
    df = df[df['psxg_gk'] == df['psxg_gk']]
  else:
    df = df[df['xg'] == df['xg']]
  #убрали сезоны с неполной статистикой

  df = df[df['comp_level'] != '1. MLS'] #убираем млс

  if (len(df['squad'].unique()) < 2):
    # print("Ошибка: Только одна команда со статистикой для " + player)
    return ([], is_keeper)
  

  # составим список всех трансферов
  cur_team = df.iloc[0,:]['squad']
  transfers = []
  for i in range(1, len(df.index)):
    new_team = df.iloc[i,:]['squad']
    if (cur_team != new_team):
      if (df.iloc[i,:]['Unnamed: 0'] - 1 == df.iloc[i - 1,:]['Unnamed: 0'] and df.iloc[i,:]['games'] >= min_games_after): #проверяем что строки соседние
        #(могла выпасть аренда в не топ5) и что число игр в новой команде достаточно большое
        if (df.iloc[i - 1,:]['games'] < min_games_before): #проверяем число игр до перехода
          if (df.iloc[i,:]['year'] == df.iloc[i - 1,:]['year'] and i > 1 and df.iloc[i - 2,:]['squad'] == cur_team): #если игр мало, но год до и после
            #перехода совпадает, и игрок прошлый сезон играл в той же команде то пробуем взять статистику за прошлый сезон
            #(могло попасть несколько игр в начале сезона, до трансфера) 
            #иногда может получится что игрок просто мало играл до зимнего трансфера, это не исправить
            if (df.iloc[i - 2,:]['games'] >= min_games_before):
              transfers.append((i - 2, i))
        else: 
          transfers.append((i - 1, i))
      cur_team = new_team

  season_tables = [] #список с датафреймами для каждого трансфера игрока
  for i, j in transfers:
    old_team = df.iloc[i,:]['squad']
    old_age = df.iloc[i,:]['age']
    new_team = df.iloc[j,:]['squad']
    new_age = df.iloc[j,:]['age']
    l = i
    while (l > 0):
      if (old_team != df.iloc[l - 1,:]['squad'] or old_age - 1 != df.iloc[l - 1,:]['age']):
        break
      l -= 1
    r = j
    while (r < len(df.index) - 1):
      if (new_team != df.iloc[r + 1,:]['squad'] or new_age + 1 != df.iloc[r + 1,:]['age']):
        break
      r += 1
    #обрежем, если взяли больше сезонов чем нужно
    l = max(l, i - max_seasons + 1)
    r = min(r, j + max_seasons - 1)
    # season_tables.append(df.iloc[l:r + 1,:])
    season_tables.append(df.iloc[l:i+1,:].append(df.iloc[j:r+1,:]))
  return (season_tables, is_keeper)

Строим списки датафреймов для каждого игрока, ограничиваем 1 годом до и после трансфера. Требуем минимум 5 игр  до и после трансфера.

In [4]:
players = list(player_stats.name.unique())
df_field = pd.DataFrame()
for player in tqdm(players):
    transfers, is_keeper = get_seasons(player, player_stats, min_games_before=5, min_games_after=5)
    if not is_keeper:
        for df in transfers:
            df_field = df_field.append(df)

# Добавление позиций, удаление дупликатов

Загрузим словарь соответствий имен игроков из таблиц fbref и transfermarkt

In [5]:
names = pd.read_csv('/content/drive/MyDrive/xg_project/dict_names.csv')
tmp = names.to_dict()
dict_names = {}
for key, value in tmp.items():
  dict_names[key] = value[0]

В парсере статистики игроков с фбрефа есть ошибка, он парсит некоторых игроков дважды: по полному и по неполному имени. Например "Edinson Roberto Cavani Gómez" и "Edinson Cavani", при этом полное имя в данном случае парсится по запросу "Roberto", что портит мэтчинг позиций. Удалим дупликаты с полным именем:

In [6]:
df_field2 = df_field.drop_duplicates()

In [7]:
dup = df_field2[df_field2.duplicated(subset = ['age', 'squad', 'minutes', 'assists_per90'], keep=False)]
# dup['name'].unique()                           

In [9]:
# for name in dup['name'].unique():
  # print(name, "---", dict_names[name])

Получили список повторяющихся имен, удалим полные

In [10]:
df_field.drop(df_field.loc[df_field['name'] == 'Pedro Antonio Porro Sauceda'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Bernardo José Espinosa Zúñiga'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Thiago Henrique Mendes Ribeiro'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Alexander Alegría Moreno'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Ander Capa Rodríguez'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Pedro Mba Obiang Avomo'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Martin Ødegaard'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Willian José da Silva'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Borja Valero Iglesias'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Alejandro Berenguer Remiro'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Pedro Bigas Rigo'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Jason David Ian Puncheon'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Allan-Roméo Nyom'].index, inplace=True)
df_field.drop(df_field.loc[df_field['name'] == 'Edinson Roberto Cavani Gómez'].index, inplace=True)

Заменим имена игроков из таблицы с fbref на имена с трансфермакта:

In [11]:
df_field.replace({'name' : dict_names}, inplace= True)

Загрузим таблицу с данными о трансферах с трансфермаркта:

In [12]:
transfers = pd.read_csv('/content/drive/MyDrive/xg_project/Transfers.csv')

Что бы не заморачиваться с мерджем просто составим словарь соответствий имен и позиций

In [13]:
positions = {}
for i, row in transfers.iterrows():
  positions[row.player_name] = row.position

Несколько ошибок исправил вручную

In [ ]:
# positions["Pedro Porro"] = "Right-Back"
# positions["Danilo"] = "Right-Back"
# positions["Ryan Sessegnon"] = "Left-Back"
# positions["Aleix Vidal"] = "Right-Back"
# positions["Arkadiusz Reca"] = "Left-Back"
# positions["Diego Farias"] = "Left Winger"
# positions["Davide Zappacosta"] = "Right-Back"
# positions["Nícolas"] = "Attacking Midfield"
# positions["Rafael"] = "Central Midfield"

In [14]:
df_field['position'] = df_field['name']
df_field.replace({'position' : positions}, inplace= True)

# Мердж таблиц

Раньше статистика игроков хранилась в виде списка датафреймов по каждому трансферу, что бы не переписывать следующие функции создадим список датафреймов

In [106]:
field = []
for i in range(0, df_field.shape[0], 2):
  field.append(df_field.iloc[i:i+2, :].copy())

Импортируем статистику клубов

In [38]:
team_stats1 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats1.csv')
team_stats2 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats2.csv')
team_stats3 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats3.csv')
team_stats4 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats4.csv')
team_stats_cadiz = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats_cadiz.csv')
team_stats_cadiz['team'] = 'Cádiz'
team_stats = pd.concat([team_stats1, team_stats2, team_stats3, team_stats4, team_stats_cadiz])

In [39]:
team_stats

,team,year,result,goals_for,goals_against,xg_for,xg_against,possession,shots_on_target_against,goals_against_gk,saves,save_pct,clean_sheets,psxg_gk,psxg_net_gk,pens_att_gk,pens_allowed,pens_saved,pens_missed_gk,passes_completed_launched_gk,passes_launched_gk,passes_pct_launched_gk,passes_gk,passes_throws_gk,pct_passes_launched_gk,passes_length_avg_gk,goal_kicks,pct_goal_kicks_launched,goal_kick_length_avg,crosses_gk,crosses_stopped_gk,crosses_stopped_pct_gk,def_actions_outside_pen_area_gk,avg_distance_def_actions_gk,goals,shots_total,shots_on_target,shots_on_target_pct,goals_per_shot,goals_per_shot_on_target,...,tackles_interceptions,clearances,errors,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,dribbles_completed_pct,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,pass_targets,passes_received,passes_received_pct,progressive_passes_received,cards_yellow,cards_red,cards_yellow_red,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Bournemouth,2017,1.157895,1.184211,1.605263,1.126316,1.542105,47.815789,4.552632,1.605263,3.000000,64.102632,0.157895,1.439474,-0.165789,0.105263,0.052632,0.026316,0.026316,4.684211,13.763158,35.047368,18.236842,4.500000,49.789474,41.618421,5.210526,82.316216,60.389189,11.421053,0.894737,8.239474,0.289474,13.300000,1.184211,12.342105,4.052632,32.536842,0.085526,0.242105,...,NaN,22.894737,0.131579,578.815789,62.342105,210.894737,243.657895,163.894737,23.342105,537.973684,8.842105,15.131579,58.850000,9.789474,0.578947,355.157895,2073.157895,1117.789474,43.342105,13.157895,4.157895,13.368421,12.210526,397.631579,341.815789,85.728947,39.973684,1.473684,0.026316,0.026316,10.131579,12.421053,1.394737,0.052632,0.000000,0.026316,105.868421,11.447368,12.763158,48.128947
1,Bournemouth,2018,1.184211,1.473684,1.842105,1.405263,1.468421,46.842105,4.368421,1.842105,2.710526,64.139474,0.263158,1.476316,-0.286842,0.131579,0.131579,0.000000,0.000000,5.552632,14.236842,40.357895,19.473684,5.078947,45.326316,40.192105,7.894737,72.302632,55.860526,9.473684,0.631579,6.092105,0.473684,14.013158,1.447368,11.421053,4.000000,34.791892,0.123243,0.319730,...,NaN,28.026316,0.157895,577.052632,66.815789,211.263158,251.947368,149.078947,25.236842,528.263158,8.473684,14.368421,57.092105,9.263158,0.447368,368.736842,1833.500000,981.078947,35.842105,12.500000,3.947368,13.184211,11.868421,423.815789,354.184211,82.863158,43.578947,1.578947,0.026316,0.000000,10.131579,12.973684,1.526316,0.210526,0.131579,0.078947,93.868421,13.789474,13.868421,50.294737
2,Bournemouth,2019,0.894737,1.052632,1.710526,1.126316,1.513158,44.052632,4.789474,1.710526,3.210526,64.191892,0.131579,1.684211,-0.026316,0.157895,0.131579,0.026316,0.000000,5.421053,17.578947,31.742105,22.605263,4.973684,53.352632,44.621053,7.815789,70.234211,58.302632,8.631579,0.526316,5.584211,0.447368,14.802632,1.000000,10.105263,3.052632,29.950000,0.088421,0.296000,...,NaN,29.868421,0.368421,544.368421,72.078947,204.763158,228.684211,141.736842,21.894737,496.789474,9.473684,17.000000,57.039474,10.394737,0.736842,320.684211,1684.473684,870.184211,32.105263,9.710526,3.263158,13.710526,11.552632,386.447368,314.289474,80.163158,36.131579,2.052632,0.078947,0.052632,11.631579,12.236842,1.789474,0.052632,0.131579,0.000000,96.263158,19.657895,21.236842,48.476316
3,Bournemouth,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sevilla,2017,1.520000,1.360000,1.520000,1.497368,1.450000,54.980000,3.920000,1.520000,2.520000,68.745652,0.340000,1

Функция для получения названий команд до и после трансфера, года последнего сезона в старой команде и окна трансфера. Летний трансфер может считатьс за зимний, если игрок успел провести больше 5 матчей за старую команду до трансфера.

In [76]:
def get_info(table):
  # возвращает (название 1 команды, название 2 команды, год начала статистики, последний год в первой команде, окно трансфера (1 если зимний))
  y = table.iloc[0]['year']
  y0 = y
  team1 = table.iloc[0]['squad']
  i = 0;
  while (table.iloc[i]['squad'] == team1):
    y = table.iloc[i]['year']
    i += 1
  team2 = table.iloc[i]['squad']
  winter = (y == table.iloc[i]['year'])
  return (team1, team2, y0, y, winter)

In [77]:
print(get_info(field[701]))

('Fiorentina', 'Atalanta', 2018.0, 2018.0, False)


Функция для получения нужных таблиц с командной статистикой

In [146]:
def get_tables(table):
  t1, t2, y0, y, is_win = get_info(table)
  df1 = team_stats[team_stats['team'] == t1]
  df1 = df1[(df1['year'] >= y0) & (df1['year'] <= y)]
  df2 = team_stats[team_stats['team'] == t2]
  df2 = df2[(df2['year'] >= y0) & (df2['year'] <= y)]
  return [df1, df2]

Новая версия, может работать не корректно если берем больше 1 сезона у каждой команды.
Если у команды нет статистики за сезон то берем за следующий

In [139]:
def get_tables(table):
  t1, t2, y0, y, is_win = get_info(table)
  df1 = team_stats[team_stats['team'] == t1]
  df1 = df1[(df1['year'] >= y0) & (df1['year'] <= y)]
  df2 = team_stats[team_stats['team'] == t2]
  df3 = df2[(df2['year'] >= y0) & (df2['year'] <= y)]
  if (df3[df3['possession'] != df3['possession']].shape[0]):
    df3 = df2[(df2['year'] >= y0 + 1) & (df2['year'] <= y + 1)]
    # print(df3['year'].iloc[0], df3['team'].iloc[0])
  return [df1, df3]

Функция Акима, которая мерджит статистику игрока и команд для каждого трансфера

In [147]:
from tqdm.notebook import tqdm

merge_table = pd.DataFrame()
for i, observation in enumerate(tqdm(field)):

    # Check there are no missing seasons
    if observation.year.values[1] - observation.year.values[0] > 1:
        continue

    old, new = get_tables(observation)
    old.columns = [x + '_old' for x in old.columns]
    new.columns = [x + '_new' for x in new.columns]

    old.reset_index(drop=True, inplace=True)
    new.reset_index(drop=True, inplace=True)

    club_features = pd.concat([old, new], axis=1)
    # club_features = pd.DataFrame(club_features.mean()).T
    club_features['year'] = club_features['year_old']
    club_features['predict_year'] = club_features['year_old']

    if get_info(observation)[-1] != 1:
        observation['predict_year'] = observation['year'] - 1
        merge_before_target = club_features.merge(observation.drop(columns='predict_year'), on=['year'], how='left')
        merge = merge_before_target.merge(observation.drop(columns='year'), on=['predict_year'], how='left')
    else:
      merge_before_target = club_features.merge(observation.iloc[:1], on=['year'], how='left')
      merge = merge_before_target.merge(observation.iloc[1:], on=['year'], how='left')

    merge_table = merge_table.append(merge)

    if merge['sca_per90_y'].isnull().sum() > 0:
      print(i, observation[['name', 'squad', 'year']])

# merge_table.drop(columns = ['year_old', 'year_new', 'predict_year'], inplace=True)

merge_table.head()

,team_old,year_old,result_old,goals_for_old,goals_against_old,xg_for_old,xg_against_old,possession_old,shots_on_target_against_old,goals_against_gk_old,saves_old,save_pct_old,clean_sheets_old,psxg_gk_old,psxg_net_gk_old,pens_att_gk_old,pens_allowed_old,pens_saved_old,pens_missed_gk_old,passes_completed_launched_gk_old,passes_launched_gk_old,passes_pct_launched_gk_old,passes_gk_old,passes_throws_gk_old,pct_passes_launched_gk_old,passes_length_avg_gk_old,goal_kicks_old,pct_goal_kicks_launched_old,goal_kick_length_avg_old,crosses_gk_old,crosses_stopped_gk_old,crosses_stopped_pct_gk_old,def_actions_outside_pen_area_gk_old,avg_distance_def_actions_gk_old,goals_old,shots_total_old,shots_on_target_old,shots_on_target_pct_old,goals_per_shot_old,goals_per_shot_on_target_old,...,losses_gk_y,clean_sheets_y,clean_sheets_pct_y,pens_att_gk_y,pens_allowed_y,pens_saved_y,pens_missed_gk_y,pens_save_pct_y,free_kick_goals_against_gk_y,corner_kick_goals_against_gk_y,own_goals_against_gk_y,psxg_gk_y,psnpxg_per_shot_on_target_against_y,psxg_net_gk_y,psxg_net_per90_gk_y,passes_completed_launched_gk_y,passes_launched_gk_y,passes_pct_launched_gk_y,passes_gk_y,passes_throws_gk_y,pct_passes_launched_gk_y,passes_length_avg_gk_y,goal_kicks_y,pct_goal_kicks_launched_y,goal_kick_length_avg_y,crosses_gk_y,crosses_stopped_gk_y,crosses_stopped_pct_gk_y,def_actions_outside_pen_area_gk_y,def_actions_outside_pen_area_per90_gk_y,avg_distance_def_actions_gk_y,dom_lg_goals_against_gk_y,dom_lg_clean_sheets_y,dom_cup_goals_against_gk_y,dom_cup_clean_sheets_y,intl_cup_goals_against_gk_y,intl_cup_clean_sheets_y,all_goals_against_gk_y,all_clean_sheets_y,position_y
0,Rennes,2017,1.526316,1.315789,1.157895,1.300000,1.410526,50.236842,3.973684,1.157895,3.000000,73.800000,0.263158,1.321053,0.215789,0.157895,0.131579,0.026316,0.000000,4.447368,13.710526,32.071053,16.947368,2.447368,55.510526,45.036842,4.763158,86.948649,63.991892,9.868421,1.105263,13.081579,0.947368,17.339474,1.263158,12.473684,3.842105,32.102632,0.103158,0.339459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Attacking Midfield
0,Saint-Étienne,2019,1.000000,1.029412,1.558824,0.985294,1.158824,49.205882,3.911765,1.558824,2.500000,58.260606,0.176471,1.402941,-0.097059,0.176471,0.147059,0.000000,0.029412,6.029412,15.764706,41.912121,22.235294,3.794118,47.944118,39.379412,7.617647,61.405882,46.767647,6.823529,0.294118,4.635294,0.411765,13.817647,0.970588,10.882353,3.294118,31.411765,0.078235,0.233939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right Winger
0,Atalanta,2020,1.777778,2.148148,1.370370,1.870370,1.244444,52.888889,3.703704,1.370370,2.333333,60.516000,0.296296,1.222222,-0.148148,0.185185,0.074074,0.111111,0.000000,5.370370,11.111111,50.644444,21.703704,4.074074,34.644444,34.470370,4.888889,64.970370,51.044444,6.555556,0.481481,5.233333,0.777778,15.011111,2.111111,15.296296,5.777778,37.829630,0.131852,0.349630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Second Striker
0,Köln,2017,0.700000,1.050000,1.950000,1.244118,2.000000,47.875000,5.625000,1.950000,3.875000,70.137500,0.150000,2.100000,0.041176,0.225000,0.200000,0.025000,0.000000,8.617647,22.205882,37.164706,22.764706,2.411765,67.044118,49.261765,8.205882,86.376471,61.352941,10.882353,0.823529,8.735294,0.970588,15.794118,1.000000,11.925000,3.750000,31.462500,0.081250,0.293590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Centre-Back
0,Udinese,2017,1.052632,1.263158,1.657895,1.147368,1.342105,44.631579,4.289474,1.657895,2.815789,62.113514,0.236842,1.315789,-0.210526,0.157895,0.052632,0.078947,0.026316

Теперь у всех трансферов есть статистика новой команды:

In [130]:
df = merge_table.loc[:, columns].dropna(subset=['possession_new'])

In [131]:
merge_table.shape, df.shape

((987, 834), (987, 16))

In [132]:
merge_table[merge_table['possession_new'] != merge_table['possession_new']].loc[:, ["team_old", "team_new", "year_old", "year_new"]]

,team_old,team_new,year_old,year_new


# Модель Акима

In [81]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [82]:
club_features   = ['possession', 'npxg', 'xg_against', 'passes_pct', 'touches']
player_features = ['minutes_90s', 'goals_per90', 'assists_per90', 'sca_per90', 'gca_per90']

In [83]:
columns = [x + '_old' for x in club_features] + [x + '_new' for x in club_features] + [x + '_x' for x in player_features] + ['sca_per90_y']

In [ ]:
[x for x in old.columns if 'xg' in x]

['xg_for_old',
 'xg_against_old',
 'psxg_gk_old',
 'psxg_net_gk_old',
 'xg_old',
 'npxg_old',
 'npxg_per_shot_old',
 'xg_net_old',
 'npxg_net_old']

In [133]:
df = merge_table.loc[:, columns].dropna(subset=['sca_per90_y', 'possession_new'])

In [134]:
train, valid_plus_test = train_test_split(df, random_state=2021, test_size=0.3)
val, test = train_test_split(valid_plus_test, random_state=2021, test_size=0.67)

In [135]:
val

,possession_old,npxg_old,xg_against_old,passes_pct_old,touches_old,possession_new,npxg_new,xg_against_new,passes_pct_new,touches_new,minutes_90s_x,goals_per90_x,assists_per90_x,sca_per90_x,gca_per90_x,sca_per90_y
0,55.631579,1.302632,1.357895,79.792105,642.000000,54.980000,1.394737,1.450000,78.518421,632.394737,29.8,0.03,0.00,0.44,0.03,0.62
0,47.875000,1.179412,2.000000,73.026471,579.323529,47.475000,1.064706,1.367647,74.917647,575.911765,14.8,0.07,0.00,2.09,0.00,1.22
0,51.447368,1.268421,1.123684,79.284211,594.710526,43.289474,1.055263,1.434211,72.239474,518.578947,19.1,0.00,0.05,0.26,0.05,0.14
0,45.647059,0.882353,1.647059,75.358824,555.382353,51.676471,1.379412,1.517647,80.155882,649.088235,11.1,0.09,0.00,0.63,0.18,0.55
0,38.842105,0.855263,1.531579,73.002632,492.578947,49.205882,0.985294,1.470588,76.988235,588.705882,8.3,0.00,0.00,1.32,0.00,0.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,44.738095,1.057143,1.478571,74.714286,539.119048,53.058824,1.341176,1.455882,82.161765,663.617647,14.9,0.33,0.07,2.81,0.47,3.31
0,58.523810,1.576190,1.090476,84.930952,783.452381,52.250000,1.955000,1.600000,79.830000,633.175000,14.1,0.14,0.14,2.13,0.28,5.29
0,43.289474,1.055263,1.434211,72.239474,518.578947,49.605263,1.118421,1.386842,75.773684,572.842105,9.3,0.00,0.00,0.86,0.00,0.23
0,50.318182,1.460526,1.234211,76.857895,583.921053,48.352941,1.232353,1.238235,72.279412,577.441176,13.9,0.07,0.00,1.96,0.58,0.52


In [136]:
target_column = 'sca_per90_y'
lgb_train = lgb.Dataset(train.drop(columns=target_column).values, train[target_column].values)
lgb_val = lgb.Dataset(val.drop(columns=target_column).values, val[target_column].values)


params = {
    "objective" : "mse",
    "metric" : "mae",
    "max_depth" : 7,
    "num_leaves" : 100,
    "learning_rate" : 0.05,
    "bagging_fraction" : 0.3,
    "feature_fraction" : 0.15,
    "lambda_l1" : 5,
    "lambda_l2" : 5,
    "bagging_seed" : 42,
    "verbosity" : 1,
    "seed": 42
}

clf = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_val],
    num_boost_round=500,
    early_stopping_rounds=10,
    verbose_eval=10
)

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's l1: 0.929757
[20]	valid_0's l1: 0.89521
[30]	valid_0's l1: 0.888074
[40]	valid_0's l1: 0.858945
[50]	valid_0's l1: 0.851181
[60]	valid_0's l1: 0.845887
[70]	valid_0's l1: 0.846803
Early stopping, best iteration is:
[61]	valid_0's l1: 0.845012


# Сплит

Пока есть небольшие проблемы с матчингом позиций и появилось несколько ошибочных вратарей.

Разделим общую таблицу на 5 частей для каждой группы позиций:

In [148]:
dict_pos = {'Attacking Midfield' : 'Wingers', 'Right Winger' : 'Wingers', 'Centre-Back' : 'Centrebacks',
       'Central Midfield' : 'Midfielders', 'Left Midfield' : 'Wingers', 'Left-Back' : 'Fullbacks', 'Right Midfield' : 'Wingers',
       'Left Winger' : 'Wingers', 'Right-Back' : 'Fullbacks', 'Centre-Forward' : 'Forwards',
       'Defensive Midfield' : 'Midfielders', 'Second Striker' : 'Forwards', 'midfield' : 'Midfielders',
       'Goalkeeper' : 'Goalkeepers'} 
tables = [t for _, t in merge_table.replace({'position_y' : dict_pos}).groupby(['position_y'])]

In [149]:
Centrebacks = tables[0]
Forwards = tables[1]
Fullbacks = tables[2]
Midfielders = tables[4]
Wingers = tables[5]

Функция для разбиения на трейн, тест и валидацию и их экспорт в csv

In [150]:
def ttv(df, name):
  train, valid_plus_test = train_test_split(df, random_state=2021, test_size=0.3)
  val, test = train_test_split(valid_plus_test, random_state=2021, test_size=0.67)
  train.to_csv('/content/drive/MyDrive/xg_project/Train Test Val/' + name + '_train.csv')
  val.to_csv('/content/drive/MyDrive/xg_project/Train Test Val/' + name + '_val.csv')
  test.to_csv('/content/drive/MyDrive/xg_project/Train Test Val/' + name + '_test.csv')

In [151]:
ttv(Centrebacks, 'Centerbacks')
ttv(Forwards, 'Forwards')
ttv(Fullbacks, 'Fullbacks')
ttv(Midfielders, 'Midfielders')
ttv(Wingers, 'Wingers')

# Функции для корректировки атрибутов на время и владение

Функция для корректировки стат на время

In [ ]:
def time_adj(df, list_of_attr):
  for attr in list_of_attr:
    df[attr] = (df[attr] * 90) / df['minutes']

Функция для корректировки стат на владение (считается что уже скорректировано на время) 

Если игрок провел в команде только половину сезона, то владение все равно берется по всему сезону

In [ ]:
def poss_adj(df, list_of_attr, on_ball): 
  # list_of_attr - список столбцов которые нужно скорректировать
  # on_ball - True для показателей на мяче (корректировка вниз при владении > 50%) и False без мяча (корр вверх)
  for i, row in df.iterrows():
    team = row['squad']
    year = row['year']
    dft = team_stats[team_stats['team'] == team]
    dft = dft[dft['year'] == year]
    dft = dft.iloc[0]
    if (dft['possession'] != dft['possession']):
      raise NotImplementedError #пока не решил что делать с комаднами без владения
    if on_ball:
      for attr in list_of_attr:
        df.loc[i, attr] = (row[attr] * 50) / dft['possession']
    else:
      for attr in list_of_attr:
        df.loc[i, attr] = (row[attr] * 50) / (100 - dft['possession'])